<a href="https://colab.research.google.com/github/Yoshiki0418/Deep_Learning/blob/main/DL_Lecture1/optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizer

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

## スクラッチ実装

In [2]:
class MLP(nn.Module):
    def __init__(self, input_dim, hid_dim, output_dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(input_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, output_dim)
        )

    def forward(self, x):
        x = self.block(x)
        return x

In [3]:
model = MLP(64, 30, 10)

In [13]:
class Optimizer():
    def __init__(self, parameters, lr=0.03):
        self.parameters = list(parameters)
        self.lr = lr

    def step(self):
        with torch.no_grad():
            for param in self.parameters:
                param -= self.lr * param.grad

    def zero_grad(self):
        for param in self.parameters:
            if param.grad is not None:
                param.grad.zero_()

In [14]:
lr = 0.01
optimizer = Optimizer(parameters=model.parameters(), lr = lr)

In [15]:
# ===データの準備====
dataset = datasets.load_digits()
data = dataset['data']
target = dataset['target']
images = dataset['images']
X_train, X_val, y_train, y_val = train_test_split(images, target, test_size=0.2, random_state=42)
X_train_mean = X_train.mean()
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / X_train_std
X_val = (X_val - X_train_mean) / X_train_std
X_train = torch.tensor(X_train.reshape(-1, 64), dtype=torch.float32)
X_val = torch.tensor(X_val.reshape(-1, 64), dtype=torch.float32)
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)
batch_size = 30
num_batches = np.ceil(len(y_train)/batch_size).astype(int)

# ログ
train_losses = []
val_losses = []
val_accuracies = []
for epoch in range(100):
    # エポック毎にデータをシャッフル
    shuffled_indices = np.random.permutation(len(y_train))
    running_loss = 0.0

    for i in range(num_batches):

        # mini batch作成
        start = i * batch_size
        end = start + batch_size

        batch_indices = shuffled_indices[start:end]
        y = y_train[batch_indices] # batch_size x 10

        X = X_train[batch_indices] # batch_size x 64
        # 順伝播と逆伝播の計算
        optimizer.zero_grad()
        preds = model(X)
        loss = F.cross_entropy(preds, y)
        loss.backward()
        running_loss += loss.item()

        # パラメータ更新
        optimizer.step()


    # validation
    with torch.no_grad():
        preds_val = model(X_val)
        val_loss = F.cross_entropy(preds_val, y_val)
        val_accuracy = torch.sum(torch.argmax(preds_val, dim=-1) == y_val) / y_val.shape[0]

    train_losses.append(running_loss/num_batches)
    val_losses.append(val_loss.item())
    val_accuracies.append(val_accuracy)
    print(f'epoch: {epoch}: train error: {running_loss/num_batches}, validation error: {val_loss.item()}, validation accuracy: {val_accuracy}')

epoch: 0: train error: 2.279960592587789, validation error: 2.167412519454956, validation accuracy: 0.18333333730697632
epoch: 1: train error: 2.0917711382110915, validation error: 1.9920040369033813, validation accuracy: 0.5249999761581421
epoch: 2: train error: 1.9026766295234363, validation error: 1.8034493923187256, validation accuracy: 0.6388888955116272
epoch: 3: train error: 1.7029969940582912, validation error: 1.6056959629058838, validation accuracy: 0.7166666388511658
epoch: 4: train error: 1.499666191637516, validation error: 1.4075322151184082, validation accuracy: 0.7666666507720947
epoch: 5: train error: 1.30664066473643, validation error: 1.225019931793213, validation accuracy: 0.8111110925674438
epoch: 6: train error: 1.1329750965038936, validation error: 1.0568445920944214, validation accuracy: 0.8416666388511658
epoch: 7: train error: 0.9831746829052767, validation error: 0.9155831933021545, validation accuracy: 0.8805555701255798
epoch: 8: train error: 0.858231320977

## torch.optim

In [16]:
from torch import optim
optimizer = optim.SGD(model.parameters(), lr=lr)